# Combine CSI and Light Data

In [1]:
import pandas as pd
import numpy as np
import glob
import datetime as dt
import os

In [2]:
def get_activity_filecount(dir, activity):
    return len(glob.glob(f"{dir}/{activity}*.csv"))

In [3]:
act_dirs = [('new_csi_data', 'new_light_data'), ('dataset/csi_data', 'dataset/light_data')]
activities = ['empty', 'sit', 'stand', 'walk']

In [4]:
for d_pair in act_dirs:
    for activity in activities:
        print(d_pair, activity, get_activity_filecount(d_pair[0], activity), get_activity_filecount(d_pair[1], activity))

('new_csi_data', 'new_light_data') empty 1 1
('new_csi_data', 'new_light_data') sit 15 15
('new_csi_data', 'new_light_data') stand 15 15
('new_csi_data', 'new_light_data') walk 15 15
('dataset/csi_data', 'dataset/light_data') empty 4 4
('dataset/csi_data', 'dataset/light_data') sit 9 9
('dataset/csi_data', 'dataset/light_data') stand 9 9
('dataset/csi_data', 'dataset/light_data') walk 9 9


In [5]:
def combine_data(fname, csi_dir, lgt_dir):
    # fname = "stand_1.csv"
    csi_df = pd.read_csv(f"{csi_dir}/{fname}").dropna(axis=1) # read in csi data and remove columns with all nulls
    lgt_df = pd.read_csv(f"{lgt_dir}/{fname}")

    # parse to pandas datetime object and floor to closest 100 millisecond
    csi_df['time'] = pd.to_datetime(csi_df['time'], unit='s', utc=True).dt.tz_convert('Asia/Kuala_Lumpur').dt.floor('100ms')
    lgt_df['msg_received_datetime'] = pd.to_datetime(lgt_df['msg_received_datetime'], format='%Y-%m-%d %H:%M:%S.%f').dt.tz_localize('Asia/Kuala_Lumpur').dt.floor('100ms')
    lgt_df['msg_published_datetime'] = pd.to_datetime(lgt_df['msg_published_datetime'], format='%Y-%m-%d %H:%M:%S.%f').dt.tz_localize('Asia/Kuala_Lumpur').dt.floor('100ms')

    # match csi start time to light start time due to issue in Raspberry Pi
    csi_min_time = csi_df['time'].min()
    lgt_min_time = lgt_df['msg_published_datetime'].min()
    csi_df['time'] = csi_df['time']  + (lgt_min_time - csi_min_time)

    # convert complex numbers to absolute values
    for idx in range(0, 64):
        csi_col = f'_{idx}'
        csi_df[csi_col] = csi_df[csi_col].apply(lambda x: np.abs(np.complex128(x)))

    # convert light data from long to wide format
    LIGHT_COLNAMES = ['sensor_' + str (i) for i in range(1,10)]
    lgt_wide_df = lgt_df.pivot_table(index="msg_published_datetime", columns="sensor_id", values="sensor_val", aggfunc="mean").add_prefix("sensor_").reset_index()
    # fill null values with previous data, then subsequent data if needed
    lgt_wide_df.loc[:, LIGHT_COLNAMES] =  lgt_wide_df.loc[:, LIGHT_COLNAMES].ffill().bfill()

    # merge csi and light data on timestamp
    merged_df = csi_df.merge(lgt_wide_df, how='inner', left_on='time', right_on='msg_published_datetime')
    return merged_df

In [6]:
def batch_rows(df, rows_per_group):
    CSI_COLNAMES = [f'_{idx}' for idx in range(0, 64)]
    LIGHT_COLNAMES = ['sensor_' + str (i) for i in range(1,10)]

    res = []
    total_groups = df.shape[0]//rows_per_group
    count = 0
    while count < total_groups:
        temp = df.iloc[count*rows_per_group:(count+1)*rows_per_group, :]
        temp = temp[CSI_COLNAMES+LIGHT_COLNAMES].values
        if not np.isnan(np.min(temp)):
            res.append(temp)
        count += 1
    return res

In [7]:
# process second data
d_pair = ('new_csi_data', 'new_light_data')
for activity in activities:
    counter = 0
    total_files = get_activity_filecount(d_pair[0], activity)
    for i in range(1, total_files+1):
        fname = f'{activity}_{i}.csv'
        try:
            res = combine_data(fname, d_pair[0], d_pair[1])
        except:
            continue
        batched_res = batch_rows(res, 100)
        for batch in batched_res:
            np.savetxt(f"230510_combined_data/{activity}_{counter}.csv", batch, delimiter=",")
            counter += 1
        print(f"dirs: {d_pair}, file name: {fname}, file shape: {res.shape}, total output files: {len(batched_res)}")

dirs: ('new_csi_data', 'new_light_data'), file name: empty_1.csv, file shape: (2167, 76), total output files: 21
dirs: ('new_csi_data', 'new_light_data'), file name: sit_1.csv, file shape: (328, 76), total output files: 3
dirs: ('new_csi_data', 'new_light_data'), file name: sit_2.csv, file shape: (330, 76), total output files: 3
dirs: ('new_csi_data', 'new_light_data'), file name: sit_3.csv, file shape: (324, 76), total output files: 3
dirs: ('new_csi_data', 'new_light_data'), file name: sit_4.csv, file shape: (342, 76), total output files: 3
dirs: ('new_csi_data', 'new_light_data'), file name: sit_5.csv, file shape: (326, 76), total output files: 3
dirs: ('new_csi_data', 'new_light_data'), file name: sit_6.csv, file shape: (318, 76), total output files: 3
dirs: ('new_csi_data', 'new_light_data'), file name: sit_7.csv, file shape: (320, 76), total output files: 3
dirs: ('new_csi_data', 'new_light_data'), file name: sit_8.csv, file shape: (324, 76), total output files: 3
dirs: ('new_csi

In [8]:
def combine_data(fname, csi_dir, lgt_dir):
    # fname = "stand_1.csv"
    csi_df = pd.read_csv(f"{csi_dir}/{fname}").dropna(axis=1) # read in csi data and remove columns with all nulls
    lgt_df = pd.read_csv(f"{lgt_dir}/{fname}", header=None, names=['msg_received_datetime', 'msg_published_datetime', 'sensor_id', 'sensor_val'])

    # parse to pandas datetime object and floor to closest 100 millisecond
    csi_df['time'] = pd.to_datetime(csi_df['time'], unit='s', utc=True).dt.tz_convert('Asia/Kuala_Lumpur').dt.floor('100ms')
    lgt_df['msg_received_datetime'] = pd.to_datetime(lgt_df['msg_received_datetime'], format='%Y-%m-%d %H:%M:%S.%f').dt.tz_localize('Asia/Kuala_Lumpur').dt.floor('100ms')
    lgt_df['msg_published_datetime'] = pd.to_datetime(lgt_df['msg_published_datetime'], format='%Y-%m-%d %H:%M:%S.%f').dt.tz_localize('Asia/Kuala_Lumpur').dt.floor('100ms')

    # match csi start time to light start time due to issue in Raspberry Pi
    csi_min_time = csi_df['time'].min()
    lgt_min_time = lgt_df['msg_published_datetime'].min()
    csi_df['time'] = csi_df['time']  + (lgt_min_time - csi_min_time)

    # convert complex numbers to absolute values
    for idx in range(0, 64):
        csi_col = f'_{idx}'
        csi_df[csi_col] = csi_df[csi_col].apply(lambda x: np.abs(np.complex128(x)))

    # convert light data from long to wide format
    LIGHT_COLNAMES = ['sensor_' + str (i) for i in range(1,10)]
    lgt_wide_df = lgt_df.pivot_table(index="msg_published_datetime", columns="sensor_id", values="sensor_val", aggfunc="mean").add_prefix("sensor_").reset_index()
    # fill null values with previous data, then subsequent data if needed
    try:
        lgt_wide_df.loc[:, LIGHT_COLNAMES] =  lgt_wide_df.loc[:, LIGHT_COLNAMES].ffill().bfill()
    except KeyError as e:
        raise Exception(f"Light sensor error: {e}") #f"Empty sensor file {lgt_dir}/{fname} due to error")
    # merge csi and light data on timestamp
    merged_df = csi_df.merge(lgt_wide_df, how='inner', left_on='time', right_on='msg_published_datetime')
    return merged_df

In [9]:
# process first data
d_pair = ('dataset/csi_data', 'dataset/light_data')
for activity in activities:
    counter = get_activity_filecount('230510_combined_data', activity)
    total_files = get_activity_filecount(d_pair[0], activity)
    for i in range(1, total_files+1):
        fname = f'{activity}_{i}.csv'
        try:
            res = combine_data(fname, d_pair[0], d_pair[1])
        except Exception as e:
            print(e)
            continue
        batched_res = batch_rows(res, 100)
        for batch in batched_res:
            np.savetxt(f"230510_combined_data/{activity}_{counter}.csv", batch, delimiter=",")
            counter += 1
        print(f"dirs: {d_pair}, file name: {fname}, file shape: {res.shape}, total output files: {len(batched_res)}")

dirs: ('dataset/csi_data', 'dataset/light_data'), file name: empty_1.csv, file shape: (272, 76), total output files: 2
dirs: ('dataset/csi_data', 'dataset/light_data'), file name: empty_2.csv, file shape: (272, 76), total output files: 2
dirs: ('dataset/csi_data', 'dataset/light_data'), file name: empty_3.csv, file shape: (269, 76), total output files: 2
dirs: ('dataset/csi_data', 'dataset/light_data'), file name: empty_4.csv, file shape: (273, 76), total output files: 2
dirs: ('dataset/csi_data', 'dataset/light_data'), file name: sit_1.csv, file shape: (267, 76), total output files: 2
dirs: ('dataset/csi_data', 'dataset/light_data'), file name: sit_2.csv, file shape: (270, 76), total output files: 2
dirs: ('dataset/csi_data', 'dataset/light_data'), file name: sit_3.csv, file shape: (271, 76), total output files: 2
dirs: ('dataset/csi_data', 'dataset/light_data'), file name: sit_4.csv, file shape: (268, 76), total output files: 2
dirs: ('dataset/csi_data', 'dataset/light_data'), file n